In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install transformers==3.0.1
!pip install torch

In [ ]:
!nvidia-smi

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
from torch.utils.data import TensorDataset, random_split
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
import os
import transformers
from transformers import BertForSequenceClassification, AdamW, BertConfig

In [ ]:
from google.colab import files
src = list(files.upload().values())[0]
open('file1.py','wb')

Saving tokenization_kobert.py to tokenization_kobert.py


<_io.BufferedWriter name='file1.py'>

In [ ]:
import tokenization_kobert
model_version = 'monologg/kobert'
tokenizer = tokenization_kobert.KoBertTokenizer.from_pretrained(model_version)

Downloading:   0%|          | 0.00/371k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/77.8k [00:00<?, ?B/s]

In [ ]:
print(tokenizer.tokenize('박주영 첫 골 도움 아르샤빈, 아스날과 결별'))

['▁박', '주', '영', '▁첫', '▁골', '▁도움', '▁아', '르', '샤', '빈', ',', '▁아', '스', '날', '과', '▁결별']


In [ ]:
tokens=['아스날','박주영','아르샤빈']

In [ ]:
# 추가된 단어의 개수 확인
num_added_toks = tokenizer.add_tokens(tokens)
num_added_toks


3

In [ ]:
# 추가된 단어의 토큰 번호 확인
encoded_entity = tokenizer.convert_tokens_to_ids(tokens)

In [ ]:
encoded_entity

[8002, 8003, 8004]

In [ ]:
print(tokenizer.tokenize('박주영 첫 골 도움 아르샤빈, 아스날과 결별'))

['박주영', '▁첫', '▁골', '▁도움', '아르샤빈', '▁', ',', '아스날', '▁', '과', '▁결별']


In [ ]:
##GPU 사용 시
device = torch.device(0)

In [ ]:
from transformers import BertForMaskedLM, AdamW, BertConfig

model = BertForMaskedLM.from_pretrained(model_version, # Use the 12-layer BERT model, with an uncased vocab.
    output_attentions = True, # Whether the model returns attentions weights.
    output_hidden_states = True, # Whether the model returns all hidden-states.    
)
model.resize_token_embeddings(len(tokenizer))

model.cuda(device)

In [ ]:
import copy
param_list=[]
name_list=[]
for name, param in model.named_parameters():
  param_list.append(copy.deepcopy(param))
  name_list.append(name)

In [ ]:
len(param_list)

204

In [ ]:
name_list[0]

'bert.embeddings.word_embeddings.weight'

In [ ]:
param_list[0]

Parameter containing:
tensor([[-0.0086,  0.0153,  0.0338,  ..., -0.0012,  0.0861, -0.0199],
        [ 0.0014,  0.0273,  0.1173,  ...,  0.0422, -0.0582, -0.0720],
        [ 0.0035, -0.0004, -0.0004,  ...,  0.0031,  0.0103, -0.0033],
        ...,
        [-0.0287,  0.0017,  0.0496,  ..., -0.0023,  0.0141,  0.0168],
        [ 0.0070, -0.0141,  0.0199,  ...,  0.0017,  0.0113, -0.0008],
        [ 0.0005, -0.0276, -0.0281,  ..., -0.0107, -0.0009, -0.0112]],
       device='cuda:0', requires_grad=True)

In [ ]:
sentences=['아르샤빈은 아스날에서 최고의 선수다','아르샤빈보다 박주영이 훨씬 잘한다']

In [ ]:
sentences

['아르샤빈은 아스날에서 최고의 선수다', '아르샤빈보다 박주영이 훨씬 잘한다']

In [ ]:
# Print the original sentence.
print(' Original: ', sentences[0])

# Print the sentence split into tokens.
print('Tokenized: ', tokenizer.tokenize(sentences[0]))

# Print the sentence mapped to token ids.
print('Token IDs: ', tokenizer.convert_tokens_to_ids(tokenizer.tokenize(sentences[0])))

 Original:  아르샤빈은 아스날에서 최고의 선수다
Tokenized:  ['아르샤빈', '▁', '은', '아스날', '▁', '에서', '▁최고의', '▁선수', '다']
Token IDs:  [8004, 517, 7086, 8002, 517, 6903, 4524, 2753, 5782]


In [ ]:
# 마스킹 하기전 모든 데이터를 숫자ID로 변경하는 작업 (가장 오래걸리는 작업)
from tqdm.notebook import tqdm_notebook
# Tokenize all of the sentences and map the tokens to thier word IDs.
input_ids = []
attention_masks = []

# For every sentence...
for sent in tqdm_notebook(sentences):
    # `encode_plus` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    #   (5) Pad or truncate the sentence to `max_length`
    #   (6) Create attention masks for [PAD] tokens.
    encoded_dict = tokenizer.encode_plus(
                        sent,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = 128,           # Pad & truncate all sentences.
                        pad_to_max_length = True,
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',
                        truncation=True# Return pytorch tensors.
                   )
    
    # Add the encoded sentence to the list.    
    input_ids.append(encoded_dict['input_ids'])
    
    # And its attention mask (simply differentiates padding from non-padding).
    attention_masks.append(encoded_dict['attention_mask'])

# Convert the lists into tensors.
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)

# Print sentence 0, now as a list of IDs.
print('Original: ', sentences[0])
print('Token IDs:', input_ids[0])

  0%|          | 0/2 [00:00<?, ?it/s]

Original:  아르샤빈은 아스날에서 최고의 선수다
Token IDs: tensor([   2, 8004,  517, 7086, 8002,  517, 6903, 4524, 2753, 5782,    3,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1])


In [ ]:
print('Original: ', sentences[1])
print('Token IDs:', input_ids[1])

Original:  아르샤빈보다 박주영이 훨씬 잘한다
Token IDs: tensor([   2, 8004, 2368, 8003, 3647, 5187, 3942, 7831,    3,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1])


In [ ]:
input_ids

tensor([[   2, 8004,  517, 7086, 8002,  517, 6903, 4524, 2753, 5782,    3,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1],
        [   2, 8004, 2368, 8003, 3647, 5187, 3942, 7831,    3,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1, 

In [ ]:
import copy
input_id  = copy.deepcopy(input_ids[0].numpy().reshape(1,128))

In [ ]:
input_id

array([[   2, 8004,  517, 7086, 8002,  517, 6903, 4524, 2753, 5782,    3,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1]])

In [ ]:
input_id[0,1]=4

In [ ]:
random_ids = []
random_ids.append(torch.from_numpy(input_id))

In [ ]:
import copy
input_id  = copy.deepcopy(input_ids[1].numpy().reshape(1,128))


In [ ]:
input_id 

array([[   2, 8004, 2368, 8003, 3647, 5187, 3942, 7831,    3,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1]])

In [ ]:
input_id[0,3]=4

In [ ]:
random_ids.append(torch.from_numpy(input_id))

In [ ]:
random_ids=torch.cat(random_ids, dim=0)

In [ ]:
random_ids

tensor([[   2,    4,  517, 7086, 8002,  517, 6903, 4524, 2753, 5782,    3,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1],
        [   2, 8004, 2368,    4, 3647, 5187, 3942, 7831,    3,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1, 

In [ ]:
from sklearn.model_selection import train_test_split
train_ids, val_ids=train_test_split(random_ids,test_size=0.5,
                                                  random_state=3333)
train_att, val_att=train_test_split(attention_masks, test_size=0.5,
                                                  random_state=3333)
train_input, val_input=train_test_split(input_ids, test_size=0.5,
                                                  random_state=3333)

In [ ]:
from torch.utils.data import TensorDataset, random_split

# Combine the training inputs into a TensorDataset.
train_dataset = TensorDataset(train_ids, train_att, train_input)
val_dataset = TensorDataset(val_ids, val_att, val_input)

# Create a 90-10 train-validation split.

# Calculate the number of samples to include in each set.
#train_size = int(0.9 * len(dataset))
#val_size = len(dataset) - train_size

# Divide the dataset by randomly selecting samples.
#train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

print('{:>5,} training samples'.format(len(train_dataset)))
print('{:>5,} validation samples'.format(len(val_dataset)))


    1 training samples
    1 validation samples


In [ ]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

# The DataLoader needs to know our batch size for training, so we specify it 
# here. For fine-tuning BERT on a specific task, the authors recommend a batch 
# size of 16 or 32.
batch_size = 26

# Create the DataLoaders for our training and validation sets.
# We'll take training samples in random order. 
train_dataloader = DataLoader(
            train_dataset,  # The training samples.
            sampler = RandomSampler(train_dataset), # Select batches randomly
            batch_size = batch_size # Trains with this batch size.
        )

# For validation the order doesn't matter, so we'll just read them sequentially.
validation_dataloader = DataLoader(
            val_dataset, # The validation samples.
            sampler = SequentialSampler(val_dataset), # Pull out batches sequentially.
            batch_size = batch_size # Evaluate with this batch size.
        )

In [ ]:
# Note: AdamW is a class from the huggingface library (as opposed to pytorch) 
# I believe the 'W' stands for 'Weight Decay fix"
optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # args.learning_rate - default is 5e-5, our notebook had 2e-5
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                )

In [ ]:
from transformers import get_linear_schedule_with_warmup

# Number of training epochs. The BERT authors recommend between 2 and 4. 
# We chose to run for 4, but we'll see later that this may be over-fitting the
# training data.
epochs = 1000


# Total number of training steps is [number of batches] x [number of epochs]. 
# (Note that this is not the same as the number of training samples).
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)

In [ ]:
import time
import datetime

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [ ]:
import random
import os
import pickle as pkl
import shutil
from transformers import AutoTokenizer
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning) 
import pandas as pd
import copy
from tqdm.notebook import tqdm_notebook

In [ ]:
# This training code is based on the `run_glue.py` script here:
# https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L128

# Set the seed value all over the place to make this reproducible.
seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# We'll store a number of quantities such as training and validation loss, 
# validation accuracy, and timings.
training_stats = []

# Measure the total training time for the whole run.
total_t0 = time.time()

# attention weight saving
## attention_train = []
## attention_valid = []

# input_id saving
## input_id_train = []
## input_id_valid = []

# Saving best-practices: if you use defaults names for the model, you can reload it using from_pretrained()

out_dir = '/content/drive/MyDrive/word_emb/'
output_dir = out_dir + 'random_all'

# Create output directory if needed
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

In [ ]:
def save_models(out_dir):
    
    # Saving best-practices: if you use defaults names for the model, you can reload it using from_pretrained()

    output_dir = out_dir

    # Create output directory if needed
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    print("Saving model to %s" % output_dir)

    # Save a trained model, configuration and tokenizer using `save_pretrained()`.
    # They can then be reloaded using `from_pretrained()`
    model_to_save = model.module if hasattr(model, 'module') else model  # Take care of distributed/parallel training
    model_to_save.save_pretrained(output_dir)
    tokenizer.save_pretrained(output_dir)

    # Good practice: save your training arguments together with the trained model
    # torch.save(args, os.path.join(output_dir, 'training_args.bin'))

In [ ]:
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    # Perform one full pass over the training set.

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # Measure how long the training epoch takes.
    t0 = time.time()

    # Reset the total loss for this epoch.
    total_train_loss = 0

    # Put the model into training mode. Don't be mislead--the call to 
    # `train` just changes the *mode*, it doesn't *perform* the training.
    # `dropout` and `batchnorm` layers behave differently during training
    # vs. test (source: https://stackoverflow.com/questions/51433378/what-does-model-train-do-in-pytorch)
    model.train()

    # For each batch of training data...
    for step, batch in enumerate(train_dataloader):

        # Progress update every 40 batches.
        if step % 40 == 0 and not step == 0:
            # Calculate elapsed time in minutes.
            elapsed = format_time(time.time() - t0)
            
            # Report progress.
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using the 
        # `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        
        # Always clear any previously calculated gradients before performing a
        # backward pass. PyTorch doesn't do this automatically because 
        # accumulating the gradients is "convenient while training RNNs". 
        # (source: https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch)
        model.zero_grad()        

        # Perform a forward pass (evaluate the model on this training batch).
        # The documentation for this `model` function is here: 
        # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
        # It returns different numbers of parameters depending on what arguments
        # arge given and what flags are set. For our useage here, it returns
        # the loss (because we provided labels) and the "logits"--the model
        # outputs prior to activation.
        loss, logits, attention_t, t_h_state = model(b_input_ids, 
                                         token_type_ids=None, 
                                         attention_mask=b_input_mask,
                                         masked_lm_labels=b_labels)
        #print('train_loss:', loss)
        
        total_train_loss += loss.item()

        # Perform a backward pass to calculate the gradients.
        loss.backward()

        # Clip the norm of the gradients to 1.0.
        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # Update parameters and take a step using the computed gradient.
        # The optimizer dictates the "update rule"--how the parameters are
        # modified based on their gradients, the learning rate, etc.
        optimizer.step()

        # Update the learning rate.
        scheduler.step()

    # Calculate the average loss over all of the batches.
    avg_train_loss = total_train_loss / len(train_dataloader)            
    
    # Measure how long this epoch took.
    training_time = format_time(time.time() - t0)

    epoch_dir = out_dir + 'random_all_epoch_' + str(epoch_i+1) + '/'
    save_models(epoch_dir)
        
    epoch_output = epoch_dir + 'random_all_weight/'      
    shutil.copytree(output_dir,epoch_output)

    tmp_list=[]
    p_list=[]
    for name, param in model.named_parameters():
      p_list.append(copy.deepcopy(param))
      tmp_list.append(name)
    
    print(p_list[0])

    print("")
    print("  Average training loss: {0:.5f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(training_time))
        

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
       device='cuda:0', requires_grad=True)

  Average training loss: 0.05262
  Training epcoh took: 0:00:00

======== Epoch 431 / 1000 ========
Training...
Saving model to /content/drive/MyDrive/word_emb/random_all_epoch_431/
Parameter containing:
tensor([[-0.0081,  0.0162,  0.0321,  ..., -0.0020,  0.0840, -0.0174],
        [ 0.0018,  0.0279,  0.1172,  ...,  0.0419, -0.0582, -0.0723],
        [ 0.0044, -0.0010,  0.0005,  ...,  0.0042,  0.0128, -0.0054],
        ...,
        [-0.0305,  0.0006,  0.0516,  ..., -0.0017,  0.0155,  0.0165],
        [ 0.0078, -0.0136,  0.0185,  ...,  0.0008,  0.0096,  0.0010],
        [-0.0016, -0.0290, -0.0267,  ..., -0.0094,  0.0006, -0.0132]],
       device='cuda:0', requires_grad=True)

  Average training loss: 0.04789
  Training epcoh took: 0:00:00

======== Epoch 432 / 1000 ========
Training...
Saving model to /content/drive/MyDrive/word_emb/random_all_epoch_432/
Parameter containing:
tensor([[-0.0081,  0.0162,  0.03


KeyboardInterrupt



In [ ]:
param_list[0]

Parameter containing:
tensor([[-0.0086,  0.0153,  0.0338,  ..., -0.0012,  0.0861, -0.0199],
        [ 0.0014,  0.0273,  0.1173,  ...,  0.0422, -0.0582, -0.0720],
        [ 0.0035, -0.0004, -0.0004,  ...,  0.0031,  0.0103, -0.0033],
        ...,
        [-0.0287,  0.0017,  0.0496,  ..., -0.0023,  0.0141,  0.0168],
        [ 0.0070, -0.0141,  0.0199,  ...,  0.0017,  0.0113, -0.0008],
        [ 0.0005, -0.0276, -0.0281,  ..., -0.0107, -0.0009, -0.0112]],
       device='cuda:0', requires_grad=True)

In [ ]:
name_list[0]

'bert.embeddings.word_embeddings.weight'

In [ ]:
from transformers import BertModel, DistilBertModel
f_model = BertModel.from_pretrained("/content/drive/MyDrive/word_emb/random_all_epoch_600")

In [ ]:
with torch.no_grad():
    f_emb= f_model(input_ids[0:1], attention_masks[0:1])
    hidden_states = f_emb[2]

In [ ]:
input_ids[0:1]

In [ ]:
print ("Number of layers:", len(hidden_states), "  (initial embeddings + 12 BERT layers)")
layer_i = 0

print ("Number of batches:", len(hidden_states[layer_i]))
batch_i = 0

print ("Number of tokens:", len(hidden_states[layer_i][batch_i]))
token_i = 0

print ("Number of hidden units:", len(hidden_states[layer_i][batch_i][token_i]))

Number of layers: 13   (initial embeddings + 12 BERT layers)
Number of batches: 1
Number of tokens: 128
Number of hidden units: 768


In [ ]:
f_emb[1]

In [ ]:
torch.mean(hidden_states[-2][0], dim=0)

In [ ]:
f_param_list=[]
f_name_list=[]
for name, param in f_model.named_parameters():
  f_param_list.append(param)
  f_name_list.append(name)

In [ ]:
f_param_list[0]

Parameter containing:
tensor([[-0.0112,  0.0140,  0.0319,  ..., -0.0036,  0.0869, -0.0174],
        [ 0.0014,  0.0275,  0.1172,  ...,  0.0425, -0.0585, -0.0722],
        [ 0.0061,  0.0009,  0.0012,  ...,  0.0057,  0.0102, -0.0070],
        ...,
        [ 0.0003, -0.0146, -0.0772,  ...,  0.0907, -0.1730,  0.0090],
        [ 0.0343,  0.0100,  0.0036,  ..., -0.0124,  0.0025, -0.0087],
        [-0.0022,  0.0037, -0.0062,  ...,  0.0034,  0.0162, -0.0065]],
       requires_grad=True)

In [ ]:
for name, param in model.named_parameters():
	if 'classifier' not in name: # classifier layer
		param.requires_grad = False

In [ ]:
model.init_weights()

In [ ]:
model.named_parameters

In [ ]:
for name, param in model.named_parameters():
  print(name)

In [ ]:
param_list=[]
name_list=[]
for name, param in model.named_parameters():
  param_list.append(param)
  name_list.append(name)

In [ ]:
model.init_weights()

In [ ]:
name_list[0]

In [ ]:
len(param_list[0])

In [ ]:
name_list[5]

In [ ]:
from transformers import BertModel, DistilBertModel
f_model = BertModel.from_pretrained("/content/drive/MyDrive/word_emb/random_all_epoch_600")
f_model.embeddings.word_embeddings

Embedding(8005, 768, padding_idx=1)

In [ ]:
f_param_list=[]
f_name_list=[]
for name, param in f_model.named_parameters():
  f_param_list.append(param)
  f_name_list.append(name)

In [ ]:
f_param_list[0]

Parameter containing:
tensor([[-0.0081,  0.0163,  0.0320,  ..., -0.0021,  0.0839, -0.0172],
        [ 0.0018,  0.0281,  0.1172,  ...,  0.0419, -0.0582, -0.0723],
        [ 0.0045, -0.0011,  0.0005,  ...,  0.0042,  0.0129, -0.0055],
        ...,
        [-0.0307,  0.0006,  0.0518,  ..., -0.0017,  0.0156,  0.0166],
        [ 0.0078, -0.0135,  0.0184,  ...,  0.0008,  0.0095,  0.0011],
        [-0.0018, -0.0292, -0.0267,  ..., -0.0094,  0.0006, -0.0133]],
       requires_grad=True)

In [ ]:
import tensorflow as tf
from numpy import dot 
from numpy.linalg import norm 
import numpy as np

# 코사인 유사도를 구하는 함수 
def cos_sim(a, b): 
  return dot(a, b)/(norm(a)*norm(b))


In [ ]:
for num, i in enumerate(range(0, 8004)):
  cos=cos_sim(param_list[0][i].detach().cpu().numpy(),f_param_list[0][i].detach().cpu().numpy())
  print(num,cos)

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
3004 0.99977154
3005 0.99968356
3006 0.99958706
3007 0.9996259
3008 0.9994348
3009 0.99970615
3010 0.9995294
3011 0.9998649
3012 0.99971455
3013 0.9998147
3014 0.9990761
3015 0.9997417
3016 0.99931794
3017 0.9994746
3018 0.9998665
3019 0.99965686
3020 0.99878573
3021 0.99937594
3022 0.99974257
3023 0.9990803
3024 0.9996056
3025 0.999511
3026 0.99974275
3027 0.9995475
3028 0.999832
3029 0.9994875
3030 0.99950886
3031 0.9991914
3032 0.9994637
3033 0.9993948
3034 0.99957067
3035 0.9995641
3036 0.9999347
3037 0.99961144
3038 0.99979866
3039 0.99959666
3040 0.9997387
3041 0.9997512
3042 0.99983335
3043 0.9995544
3044 0.999616
3045 0.9996791
3046 0.9996517
3047 0.9996813
3048 0.99964124
3049 0.9991642
3050 0.99948
3051 0.99987876
3052 0.9998118
3053 0.9996978
3054 0.9996654
3055 0.99960005
3056 0.9996893
3057 0.99957937
3058 0.9997975
3059 0.9992883
3060 0.9994528
3061 0.99966884
3062 0.9998311
3063 0.9997609
3064 0.999803
3065 0.9994939
3066 0.99978507
30

In [ ]:
f_param_list[0][0].detach().numpy()